torch.nn : 신경망을 구성하는데 필요한 모든 구성요소를 제공  

PyTorch의 모든 모듈은 nn.Model의 하위 클래스  
신경망은 다른 모듈로 구성된 모듈

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# GPU 사용

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


신경망 모델을 nn.Module의 하위 클래스로 정의  
__ init __ 에서 신경망 계층들을 초기화  
nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현합니다.

In [11]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork,self).__init__() #이건 왜하지?
    self.flatten = nn.Flatten()
    self.linear_reulu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
        nn.ReLU()
    )
  
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_reulu_stack(x)
    return logits

NeuralNetwork의 인스턴스를 생성하고, device로 이동한뒤, 구조 출력

In [12]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_reulu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


일부 백그라운드 연산들과 함께 모델의 forward를 실행합니다.  
model.forward()를 직접 호출시키지마라!  
(강의에서는 직접 호출시키던데...)

모델에 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 10 차원 텐서가 반환  
원시 예측값을 nn.Sofrtmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

In [13]:
X = torch.rand(1,28,28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


#모델 계층(Layer)

In [14]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#nn.Flatten

dim=0의 미니배치 차원은 유지됨.

In [17]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#nn.Linear  

선형 계층은 저장된 가중치(weight)과 편향(bias)을 사용하여  
입력에 선형 변환을 적용하는 모듈

In [19]:
layer1 = nn.Linear(in_features=28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#nn.ReLU  

비선형 활성화는 모델의 입력과 출력 사이에 복잡한 관계(mapping)을 만듭니다.  
비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity)을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.  

ReLU 말고도 다른 것을 사용할 수 있습니다.

In [21]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU : {hidden1}")

Before ReLU: tensor([[ 0.0696,  0.1111,  0.0101,  0.4592,  0.1485,  0.0433, -0.2587, -0.3178,
         -0.3945,  0.7651, -0.3735, -0.2132,  0.1938, -0.0760,  0.1627,  0.5133,
         -0.3048,  0.3337, -0.4908, -0.4277],
        [ 0.0566, -0.0500,  0.0793,  0.4320,  0.1004,  0.2827, -0.0604, -0.0658,
         -0.4238,  0.4063, -0.6528,  0.0309,  0.2226, -0.4336, -0.1983,  0.3539,
         -0.0908,  0.4552, -0.2047, -0.3597],
        [ 0.2656, -0.1672, -0.1596,  0.5624, -0.1646,  0.1682, -0.1662, -0.2291,
         -0.4888,  0.5328, -0.1079, -0.0212,  0.2637, -0.1313, -0.1582,  0.1906,
          0.0383,  0.2418,  0.2074, -0.3390]], grad_fn=<AddmmBackward>)


After ReLU : tensor([[0.0696, 0.1111, 0.0101, 0.4592, 0.1485, 0.0433, 0.0000, 0.0000, 0.0000,
         0.7651, 0.0000, 0.0000, 0.1938, 0.0000, 0.1627, 0.5133, 0.0000, 0.3337,
         0.0000, 0.0000],
        [0.0566, 0.0000, 0.0793, 0.4320, 0.1004, 0.2827, 0.0000, 0.0000, 0.0000,
         0.4063, 0.0000, 0.0309, 0.2226, 0.0000, 0.00

#nn.Sequential  

순서를 갖는 모듈의 컨테이너  
데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됨.  
순차 컨테이너를 사용하여 아래의 seq_models와 같은 신경망을 빠르게 만들 수 있습니다.

In [23]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#nn.Softmax

신경망의 마지막 선형 계층 nn.Softmax
[-inf,inf[ 범위의 원시값 (raw value)인 logits을 반환  
logits은 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정됨.  

dim 매개변수는 값의 합이 1이 되는 차원을 나타냄

In [24]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#모델 매개변수 

신경망 내부의 많은 계층들은 매개변수화 됩니다.  
즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다.  
nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됩니다.  

예제는 각 매개변수들을 순회하며, 매개변수의 크기와 값을 출력


In [28]:
print("Model structure : ",model,'\n\n')
for name, param in model.named_parameters():
  print(f"Layer:{name} | Size: {param.size()}|Values : {param[:2]} \n")

Model structure :  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_reulu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer:linear_reulu_stack.0.weight | Size: torch.Size([512, 784])|Values : tensor([[-0.0078, -0.0296,  0.0066,  ..., -0.0244, -0.0053,  0.0084],
        [-0.0058,  0.0329,  0.0009,  ...,  0.0176, -0.0144, -0.0167]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer:linear_reulu_stack.0.bias | Size: torch.Size([512])|Values : tensor([ 0.0117, -0.0144], device='cuda:0', grad_fn=<SliceBackward>) 

Layer:linear_reulu_stack.2.weight | Size: torch.Size([512, 512])|Values : tensor([[-0.0296,  0.0164, -0.0412,  ...,  0.0057,  0.0250,  0.0347],
        [ 0.0209, -0.0338,  0.0040,  ...,  0.0030, -0.0347, -0.0250]],
       device='cuda:0',